In [9]:
import os
import cv2
import mediapipe as mp

<span style="color: cyan">Kiểm tra thông số của video</span>

In [10]:
def check_frames_per_second(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Không thể mở video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")

    video_length_in_seconds = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

    print(f"Video length in seconds: {video_length_in_seconds}")

video_path = "12.mp4"
check_frames_per_second(video_path)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.001133725673775
Video length in seconds: 50


<span style="color: cyan">Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder</span>

In [11]:
# Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder
def extract_images_from_video(video_path, output_folder, time_step=0.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS) : {fps}")
    
    t = 0
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if t % int(fps * time_step) == 0:
                count += 1
                cv2.imwrite(output_folder + f"/frame_{count}.jpg", frame)
            t += 1
        else:
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [12]:
extract_images_from_video('12.mp4', 'data/images/bicep_curl_12/')

Frames per second using video.get(cv2.CAP_PROP_FPS) : 30.001133725673775


<span style="color: cyan">Từ tất cả các ảnh trong 1 folder, vẽ key point lên các ảnh rồi chuyển vào 1 folder mới</span>

In [13]:
def add_key_points_to_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Khởi tạo MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=True, model_complexity=1, smooth_landmarks=True
    )
    mp_drawing = mp.solutions.drawing_utils

    for image_name in os.listdir(input_folder):
        print(image_name)

        # Vẽ keypoint lên ảnh và lưu vào folder mới
        image_path = os.path.join(input_folder, image_name)
        image = cv2.imread(image_path)

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            pose_image = image.copy()
            mp_drawing.draw_landmarks(
                pose_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

        output_path = os.path.join(output_folder, image_name)

        cv2.imwrite(output_path, pose_image)

In [14]:
add_key_points_to_images("data/images/bicep_curl_12", "data/images/bicep_curl_pose_12")

frame_1.jpg
frame_10.jpg
frame_100.jpg
frame_101.jpg
frame_102.jpg
frame_11.jpg
frame_12.jpg
frame_13.jpg
frame_14.jpg
frame_15.jpg
frame_16.jpg
frame_17.jpg
frame_18.jpg
frame_19.jpg
frame_2.jpg
frame_20.jpg
frame_21.jpg
frame_22.jpg
frame_23.jpg
frame_24.jpg
frame_25.jpg
frame_26.jpg
frame_27.jpg
frame_28.jpg
frame_29.jpg
frame_3.jpg
frame_30.jpg
frame_31.jpg
frame_32.jpg
frame_33.jpg
frame_34.jpg
frame_35.jpg
frame_36.jpg
frame_37.jpg
frame_38.jpg
frame_39.jpg
frame_4.jpg
frame_40.jpg
frame_41.jpg
frame_42.jpg
frame_43.jpg
frame_44.jpg
frame_45.jpg
frame_46.jpg
frame_47.jpg
frame_48.jpg
frame_49.jpg
frame_5.jpg
frame_50.jpg
frame_51.jpg
frame_52.jpg
frame_53.jpg
frame_54.jpg
frame_55.jpg
frame_56.jpg
frame_57.jpg
frame_58.jpg
frame_59.jpg
frame_6.jpg
frame_60.jpg
frame_61.jpg
frame_62.jpg
frame_63.jpg
frame_64.jpg
frame_65.jpg
frame_66.jpg
frame_67.jpg
frame_68.jpg
frame_69.jpg
frame_7.jpg
frame_70.jpg
frame_71.jpg
frame_72.jpg
frame_73.jpg
frame_74.jpg
frame_75.jpg
frame_76.jpg
fra

<span style="color: cyan">Tool để gán nhãn nhanh</span>

In [15]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QPushButton
from PyQt5.QtGui import QPixmap

import os
import shutil

def copy_images_to_folder(input_file, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # nếu trùng tên file thì sẽ bị ghi đè
    shutil.copy(input_file, output_folder)


def window(input_folder, output_folder, shortcut):
    original_folder = input_folder.replace("_pose_", "_")

    app = QApplication(sys.argv)
    window = QMainWindow()
    window.setGeometry(200, 200, 1000, 800)
    window.setWindowTitle("Bicep curl")
    window.setStyleSheet("background-color: black;")

    images = os.listdir(input_folder)
    image_count = images.__len__()
    current_image_index = 0

    # Tạo label hiển thị tên ảnh
    name_pic = QLabel(window)
    name_pic.setText(images[current_image_index])
    name_pic.setStyleSheet("color: white;")
    name_pic.move(50, 700)
    name_pic.setGeometry(50, 700, 800, 50)

    # Vẽ ảnh lên màn hình cửa sổ với ảnh từ file
    pixmap = QPixmap(f"{input_folder}/{images[current_image_index]}")
    label = QLabel(window)
    label.setPixmap(pixmap)
    label.setGeometry(50, 50, 500, 600)
    label.setScaledContents(True)

    def update_ui():
        nonlocal current_image_index
        pixmap.load(f"{input_folder}/{images[current_image_index]}")
        label.setPixmap(pixmap)
        label.repaint()
        name_pic.setText(str(current_image_index + 1) + "/" + str(image_count) + " - " + images[current_image_index])

    update_ui()

    # Tạo button để hiển thị ảnh
    btn_continue = QPushButton(window)
    btn_continue.setText("Continue")
    btn_continue.setStyleSheet("background-color: white;")
    btn_continue.move(900, 700)

    # Thêm sự kiện khi click vào button
    def btn_continue_clicked():
        nonlocal current_image_index
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0
        update_ui()

    # Add event listener cho button
    btn_continue.clicked.connect(btn_continue_clicked)

    btn_back = QPushButton(window)
    btn_back.setText("Back")
    btn_back.setStyleSheet("background-color: white;")
    btn_back.move(600, 700)

    # Thêm sự kiện khi click vào button
    def btn_back_clicked():
        nonlocal current_image_index
        current_image_index -= 1
        if current_image_index < 0:
            current_image_index = len(images) - 1

        update_ui()

    btn_back.clicked.connect(btn_back_clicked)

    label_status = QLabel(window)
    label_status.setText(f"Copied image to folder using keys in: {[key for key in shortcut.keys()]}")
    label_status.setStyleSheet("color: white;")
    label_status.setGeometry(50, 750, 800, 50)

    def key_press_event(event):
        nonlocal current_image_index

        key = chr(event.key()).lower()
        catalog = shortcut.get(key, "Another folder")
        copy_images_to_folder(
            f"{original_folder}/{images[current_image_index]}",
            f"{output_folder}/{catalog}",
        )
        label_status.setText("Copied " + images[current_image_index] +  " to folder: " + catalog)

        # to next image
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0

        update_ui()

    window.keyPressEvent = key_press_event

    window.show()
    try:
        sys.exit(app.exec_())
    except:
        print("Exiting")

<span style="color: cyan">Thử nghiệm tool gán nhãn</span>

In [16]:
# Thay đổi shortcut này tuỳ vào bài toán
shortcut = {
    "u": "Up",
    "m": "Middle",
    "d": "Down"
}

window("./data/images/bicep_curl_pose_12", "./data/images/bicep_curl_pose_12selected/", shortcut)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

ValueError: chr() arg not in range(0x110000)

Exiting
